# Análisis de riesgo en activos financieros.

## Para un activo aislado: ##

Una serie temporal de precios puede definirse como: 

$$p(t) : T \to P$$

Es decir: 
$\forall t \in T , \exists p_t \in P : p(t)=p_t$
; siendo que: 
$T \subset \mathbb{N}^{*}$
y
$P\subset \mathbb{Q}$

$$P = \big\{p_t \big{/} \forall t, \exists {p_t} : p(t)=p_t\big\}$$

A partir de $P$ podemos definir $R$ como conjunto de llegada de la siguiente funcion: 

$$r(t) : T \to R$$



$$r(t) = \displaystyle \frac{p(t) - p(t-1)}{p(t-1)} = r_t$$



$$R =\bigg \{r_t : r_t = \left ( \frac{p_{t} - p_{t-1}}{p_{t-1}} \right ) = \left (\frac{p_{t}}{p_{t-1}} - 1 \right ) \bigg \}$$

nótese que el primer elemento no tiene anterior, por tanto no existe $r(t)$ para este punto, siendo así: $n(R) = n(P) - 1$

Por tanto para cumplir que 
$\forall t \in T, \exists r(t)$
debemos agregar un elemento de llegada para el primer t que sera cero: 
$r(t_0) = 0$

La función 
$r(t)$
ahora queda entonces definida: 

$$r(t) = 
  \begin{cases} 
   \ 0 & \text{si } t = 1 \\
    \frac{p(t) - p(t-1)}{p(t-1)}      & \text{si } t \neq 1
  \end{cases}$$


Asi cumpliendo 
$n(R) = n(P)$

Por todo esto podremos decir:

$$\implies \displaystyle r_t = (p_{t} - p_{t-1}) (p_{t-1})^{-1} = \displaystyle p_t (p_{t-1})^{-1}-1$$



además se verifica que:
$(r_t+1) p_{t-1} = p_t \quad \quad \forall t\neq 1$


El rendimiento promedio 
$\overline r$
para 
$R$
será:
$$\overline r = \displaystyle\frac {\displaystyle\sum_{t=1}^{n}r_t}{n}$$
Y su desvio estandar 
$\sigma_r$
será:
$$\displaystyle{\sigma_r = \sqrt{\frac{\displaystyle \sum_{t=1}^{n}(r_t- \overline r)^2}{n}}}$$

Definiremos un conjunto 
$F = \{r_{fav} \in R :r_{fav} > \alpha \}$
; siendo 
$\alpha$
un rendimiento arbitrario, por ejemplo 0 (cero) para separar rendimientos positivos de negativos, o por ejemplo 0,5% que podria ser una comisión que pagamos por operar, por tanto
$\alpha$
es lo mínimo que aceptamos como favorable. De manera análoga definiremos 
$D=\{r_{desf} \in R : r_{desf} \leq \alpha \}$
; de tal forma que: 
$F \cup D = R \implies n(R) = n(F) + n(D) \iff n = n_{fav} + n_{desf}$

De esta forma podemos definir 
$\overline r_{fav} = \displaystyle \frac {\displaystyle \sum_{t=t}^{n_{fav}}r_{fav}}{ n_{fav}}$

La dispersión en 
$F$
será: 
$\sigma_{fav} = \sqrt{\displaystyle\frac{(r_{fav}-\overline r_{fav})^2}{n_{fav}}}$

Y la ocurrencia de rendimientos favorables será:
$w^{+} = \displaystyle\frac{n_{fav}}{n}$

De forma análoga tenemos:

$$\overline r_{desf} = \displaystyle \frac{\displaystyle \sum_{t=t}^{n_{desf}}r_{desf}}{n_{desf}}$$

$$\sigma_{desf} = \sqrt{\displaystyle\frac{(r_{desf}-\overline r_{desf})^2}{n_{desf}}}$$

$$w^{-} = \displaystyle\frac{n_{desf}}{n}$$

Si pretendemos medir la "amenaza", es decir cuantificar cuan probable es la ocurrencia desfavorable, y cuan desfavorable podria llegar a ser, 
definiremos la amenaza
$\breve{a}$
como:

$$\breve{a} = w^{-}(\overline r_{desf}-\sigma_{desf})$$

El caso contrario podemos definirlo como oportunidad y será:

$$\hat o = w^{+}(\overline r_{fav}+\sigma_{fav})$$

Una buena aproximación a la cuantificación del riesgo 
$s$
es relacionar cuanto podemos llegar a perder respecto a cuanto podemos ganar, por tanto comparamos la longitud de 
$\breve a$ 
respecto a 
$\alpha$
, con la longitud de 
$\hat o$
respecto también a 
$\alpha$
. Es decir:

$$s = \displaystyle \frac{\alpha - \breve{a}}{\hat o - \alpha}$$

se cumple además:

$\breve a \leq \alpha \land \hat o > \alpha \implies s\geq 0$

A continuación una implementación en python 3 aplicando lo establecido hasta aqui.

In [27]:
import requests
import json
import pandas
import math

class AnalisisSP500(object):
        """ El siguiente ejemplo se desarrollo con los precios ajustados del indice SP500
        en timeframe diario, la muestra consta de 559 dias, los datos fueron descargados 
        de yahoo finance """

        def __init__(self):
            self.procesed = {}
            self.sesion = requests.Session()

        def getPrices(self):
            data = json.loads(self.sesion.get(
                "https://query1.finance.yahoo.com/v8/finance/chart/%5EGSPC?symbol=^GSPC&period1=1500000000&period2=1570067246&interval=1d&includePrePost=true&events=div|split|earn&lang=en-US&region=US&crumb=4RnTJN8Pn3e&corsDomain=finance.yahoo.com"
            ).text)
            serieHist = {"Precios":data["chart"]["result"][0]["indicators"]["adjclose"][0]["adjclose"]}
            i = list(range(1,len(serieHist["Precios"])+1))
            self.procesed["SP500"] = pandas.DataFrame(serieHist, index=i)
            return self.procesed["SP500"]
        
        def rendimientos(self):
            T = list(range(2,len(self.procesed["SP500"])+1))
            R = [0]
            data = self.procesed["SP500"]
            P = self.procesed["SP500"]["Precios"]
            for t in T:
                r = ((P[t]-P[t-1])/P[t-1])*100
                R.append(r)
            data["Rendimientos %"] = R
            self.procesed["rendSP500"] = data
            return data
        
        def promR(self):
            sumar = 0
            for r in self.procesed["rendSP500"]["Rendimientos %"]:
                sumar += r
            rProm = sumar/len(self.procesed["rendSP500"]["Rendimientos %"])
            self.procesed["rProm"] = rProm
            return rProm
        
        def desvioR(self):
            prom = self.procesed["rProm"]
            sumar = 0
            for r in self.procesed["rendSP500"]["Rendimientos %"]:
                #print(r)
                n = r-prom
                sumar += n*n
            desv = math.sqrt(sumar/len(self.procesed["rendSP500"]["Rendimientos %"]))
            self.procesed["desvioR"] = desv
            return desv #self.procesed["rendSP500"].std()#["Rendimientos %"].std()
        
        def favorables(self, alfa=0):
            F = []
            for r in self.procesed["rendSP500"]["Rendimientos %"]:
                if r>alfa:
                    F.append(r)
            self.procesed["favorables"] = F
            return F
        
        def desfavorables(self, alfa=0):
            D = []
            for r in self.procesed["rendSP500"]["Rendimientos %"]:
                if r<=alfa:
                    D.append(r)
            self.procesed["desfavorables"] = D
            return D
        
        def wdesfav(self):
            wdf = len(self.procesed["desfavorables"])/len(self.procesed["rendSP500"]["Rendimientos %"])
            self.procesed["wdesfav"] = wdf
            return wdf
        
        def wfav(self):
            wf = len(self.procesed["favorables"])/len(self.procesed["rendSP500"]["Rendimientos %"])
            self.procesed["wfav"] = wf
            return wf
        
        def promF(self):
            suma = 0
            for r in self.procesed["favorables"]:
                suma += r
            promFav = suma/len(self.procesed["favorables"])
            self.procesed["promFav"] = promFav
            return promFav
        
        def promD(self):
            suma = 0
            for r in self.procesed["desfavorables"]:
                suma += r
            promdf = suma/len(self.procesed["desfavorables"])
            self.procesed["promdesfav"] = promdf
            return promdf
        
        def sigmaF(self):
            suma = 0
            for r in self.procesed["favorables"]:
                suma += r**2
            sF = math.sqrt((suma/len(self.procesed["favorables"])))
            self.procesed["sigmafav"] = sF
            return sF
        
        def sigmaD(self):    
            suma = 0
            for r in self.procesed["desfavorables"]:
                suma += r**2
            sD = math.sqrt((suma/len(self.procesed["desfavorables"])))
            self.procesed["sigmadfav"] = sD
            return sD
        
        def amenaza(self):
            am = self.procesed["wdesfav"]*(self.procesed["promdesfav"]-self.procesed["sigmadfav"])
            self.procesed["amenaza"] = am
            return am
        
        def oportunidad(self):
            op = self.procesed["wfav"]*(self.procesed["promFav"]-self.procesed["sigmafav"])
            self.procesed["oportunidad"] = op
            return op
        
        def riesgoS(self,alfa=0):
            s = (alfa - self.procesed["amenaza"])/(self.procesed["oportunidad"]-alfa)
            self.procesed["riesgoS"] = s
            return s
        


In [29]:
sp = AnalisisSP500()
sp.getPrices()
sp.rendimientos()
sp.promR()
sp.desvioR()
sp.favorables()
#sp.desfavorables()
#sp.wfav()
#sp.wdesfav()
#sp.promF()
#sp.promD()
#sp.sigmaF()
#sp.sigmaD()
#sp.amenaza()
#sp.oportunidad()
#sp.riesgoS()

[0.05978573436131968,
 0.537263935608894,
 0.29231716783382333,
 0.028266394804660287,
 0.24491149692504183,
 0.04926487188865764,
 0.18891035295690772,
 0.164719978992806,
 0.12755699762128578,
 1.0043754544535421,
 0.14201029158568218,
 0.11626510699037035,
 0.9940779787754696,
 0.16728693563306823,
 0.048707205438578205,
 0.08428217368100845,
 0.46151489339618734,
 0.57209762207304,
 0.19825406825329187,
 0.3128726773688498,
 1.0839299318777185,
 0.33639480109454156,
 0.07571207193879037,
 0.18471814878801338,
 0.14559207541649977,
 0.11101954279371426,
 0.06343478158390796,
 0.06477937229520492,
 0.007216672978810851,
 0.4085143947080768,
 0.12046155961590974,
 0.3705110081443753,
 0.38740034570177684,
 0.2158838127297699,
 0.12467201713794737,
 0.5646787369921429,
 0.23224122617073772,
 0.18035072962025747,
 0.08781073155287943,
 0.17507533144689164,
 0.06725789853144679,
 0.07423349056041324,
 0.0327997894560449,
 0.5116842738986895,
 0.16179082781736198,
 0.1270946219078209,
 0.

In [13]:
sp.procesed

,Precios,Rendimientos %
1,2459.270020,0.000000
2,2459.139893,-0.005291
3,2460.610107,0.059786
4,2473.830078,0.537264
5,2473.449951,-0.015366
6,2472.540039,-0.036787
7,2469.909912,-0.106373
8,2477.129883,0.292317
9,2477.830078,0.028266
10,2475.419922,-0.097269
